# TIF y tif

In [6]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Ruta raíz donde están las carpetas con archivos .tif
ruta_raiz = "C:/Users/COMIMSA/Desktop/Capas/"

# EPSG de destino
epsg_destino = "6370"

def reproyectar_raster(archivo_tif):
    # Nombre temporal para evitar conflictos al sobrescribir
    archivo_temp = archivo_tif.replace(".tif", "_temp.tif").replace(".TIF", "_temp.TIF")
    
    with rasterio.open(archivo_tif) as src:
        transform, width, height = calculate_default_transform(
            src.crs, f"EPSG:{epsg_destino}", src.width, src.height, *src.bounds
        )
        perfil = src.profile.copy()
        perfil.update({
            "crs": f"EPSG:{epsg_destino}",
            "transform": transform,
            "width": width,
            "height": height
        })

        with rasterio.open(archivo_temp, "w", **perfil) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=f"EPSG:{epsg_destino}",
                    resampling=Resampling.nearest
                )

    # Cerrar los archivos antes de reemplazar
    os.remove(archivo_tif)  # Eliminar el original
    os.rename(archivo_temp, archivo_tif)  # Renombrar el reproyectado

# Recorrer las carpetas y procesar los archivos .tif
for carpeta_raiz, subcarpetas, archivos in os.walk(ruta_raiz):
    for archivo in archivos:
        if archivo.lower().endswith(".tif"):
            ruta_completa = os.path.join(carpeta_raiz, archivo)
            print(f"Reproyectando: {ruta_completa}")
            reproyectar_raster(ruta_completa)
            print(f"✔ {archivo} reproyectado y sobrescrito.")


Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-01_Precipitación media diaria para un periodo de retorno de 10 años\04004_ISO10D.TIF
✔ 04004_ISO10D.TIF reproyectado y sobrescrito.
Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-02_Precipitación media diaria para un periodo de retorno de 50 años\04004_ISO50D.TIF
✔ 04004_ISO50D.TIF reproyectado y sobrescrito.
Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-03_Precipitación media diaria para un periodo de retorno de 100 años\04004_ISO100D.TIF
✔ 04004_ISO100D.TIF reproyectado y sobrescrito.
Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-04_Excedente pluvial proyectado a 24 horas en un horizonte de 10 años\04004_PESC10D.TIF
✔ 04004_PESC10D.TIF reproyectado y sobrescrito.
Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-05_Excedente pluvial proyectado a 24 horas en un horizonte de 50 años\04004_PESC50D.TIF
✔ 04004_PESC50D.TIF reproyectado y sobrescrito.
Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-06_Excedente pluvial pr

In [2]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Ruta raíz donde están las carpetas con archivos .tif
ruta_raiz = "C:/Users/COMIMSA/Desktop/Capas/"

# EPSG de destino
epsg_destino = "6370"

def reproyectar_raster(archivo_tif):
    # Nombre temporal para evitar conflictos al sobrescribir
    archivo_temp = archivo_tif.replace(".tif", "_temp.tif")
    
    with rasterio.open(archivo_tif) as src:
        transform, width, height = calculate_default_transform(
            src.crs, f"EPSG:{epsg_destino}", src.width, src.height, *src.bounds
        )
        perfil = src.profile.copy()
        perfil.update({
            "crs": f"EPSG:{epsg_destino}",
            "transform": transform,
            "width": width,
            "height": height
        })

        with rasterio.open(archivo_temp, "w", **perfil) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=f"EPSG:{epsg_destino}",
                    resampling=Resampling.nearest
                )

    # Cerrar los archivos antes de reemplazar
    os.remove(archivo_tif)  # Eliminar el original
    os.rename(archivo_temp, archivo_tif)  # Renombrar el reproyectado

# Recorrer las carpetas y procesar los archivos .tif
for carpeta_raiz, subcarpetas, archivos in os.walk(ruta_raiz):
    for archivo in archivos:
        # print(archivo)
        if archivo.endswith(".tif"):
            ruta_completa = os.path.join(carpeta_raiz, archivo)
            print(f"Reproyectando: {ruta_completa}")
            reproyectar_raster(ruta_completa)
            print(f"✔ {archivo} reproyectado y sobrescrito.")


Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-14_Susceptibilidad física a incendios forestales\04004_Susceptibilidad_incendios_forestales.tif
✔ 04004_Susceptibilidad_incendios_forestales.tif reproyectado y sobrescrito.
Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-15_Peligro por incendios forestales horizonte a futuro cercano (2021-2040)\04004PIncendios_HC.tif
✔ 04004PIncendios_HC.tif reproyectado y sobrescrito.
Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-16_Peligro por incendios forestales horizonte a futuro medio (2040- 2060)\04004PIncendios_HM.tif
✔ 04004PIncendios_HM.tif reproyectado y sobrescrito.
Reproyectando: C:/Users/COMIMSA/Desktop/Capas/A-04-17_Peligro por incendios forestales horizonte a futuro lejano (2081-2100)\04004PIncendios_HL.tif
✔ 04004PIncendios_HL.tif reproyectado y sobrescrito.


## TIF y tif